In [4]:
import keyring
import getpass
import github
import datetime

In [2]:
# access_token = getpass.getpass()
# keyring.set_password("github-access-token", "radiotherapyai-bot", access_token)

In [3]:
access_token = keyring.get_password("github-access-token", "radiotherapyai-bot")

In [ ]:
url = 'http://api.github.com/graphql'

In [ ]:
json_schema_path = 'github_schema.json'

In [10]:
!poetry run python -m sgqlc.introspection \
     --exclude-deprecated \
     --exclude-description \
     -H "Authorization: bearer {access_token}" \
     {url} \
     {json_schema_path}

In [11]:
!poetry run sgqlc-codegen schema {json_schema_path} github_schema.py

In [13]:
from sgqlc.operation import Operation
from github_schema import github_schema as schema

In [47]:
from sgqlc.endpoint.http import HTTPEndpoint


headers = {'Authorization': f'bearer {access_token}'}

endpoint = HTTPEndpoint(url, headers)

In [67]:
op = Operation(schema.Query)

issues = op.repository(owner='RadiotherapyAI', name='regulatory').issues(first=2)
issues.nodes.title()

title

In [76]:
query = """query { 
  viewer { 
    login
  }
}"""

In [77]:
data = endpoint(query)

In [79]:
# data

In [68]:
op

query {
  repository(owner: "RadiotherapyAI", name: "regulatory") {
    issues(first: 2) {
      nodes {
        title
      }
    }
  }
}

In [69]:
data = endpoint(op)

In [80]:
# data

In [52]:
# data

In [53]:
repo = (op + data).repository
for issue in repo.issues.nodes:
    print(issue)

AttributeError: 'Query' object has no attribute 'repository'

In [43]:
merged = op + data

Query()

In [44]:
merged

Query()

In [26]:
data

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

In [4]:
github_instance = github.Github(access_token)

In [5]:
github_instance

In [6]:
for repo in github_instance.get_user().get_repos():
    print(repo.name)

regulatory


In [7]:
repo = github_instance.get_repo("RadiotherapyAI/regulatory")

In [8]:
repo.name

'regulatory'

In [9]:
pull_requests = list(repo.get_pulls(direction='asc'))

In [10]:
pr_with_an_approval = repo.get_pull(20)

In [11]:
reviews = list(pr_with_an_approval.get_reviews())

In [12]:
for review in reviews:
    if review.state == "APPROVED":
        print(f"{review.user.name} approved {review.commit_id} on the {review.submitted_at.strftime('%Y-%b-%d')}")

Oliver Eidel approved ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5 on the 2022-Mar-01


In [13]:
review = reviews[0]

In [14]:
review.commit_id

'ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5'

In [15]:
review.user

NamedUser(login="olieidel")

In [16]:
review.submitted_at.strftime('%Y-%b-%d')

'2022-Mar-01'

In [17]:
list()

[]

In [18]:
for a_file in pr_with_an_approval.get_files():    
    print(f"https://github.com/RadiotherapyAI/regulatory/blob/{review.commit_id}/{a_file.filename}")

https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/62304-mapping.md
https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/quality-manual-policy-objectives.md
https://github.com/RadiotherapyAI/regulatory/blob/ff93226e7b8be303161ce184ef0bbc2ae3bdcbe5/src/documents/drafts/sop-integrated-software-development.md


In [21]:
pr_with_an_approval.merged_by.name

'Simon Biggs'

In [20]:
pr_with_an_approval.merged_at

datetime.datetime(2022, 3, 23, 5, 57, 58)

In [57]:
commits = list(repo.get_commits(path='src/documents/released/sop-document-record-control/index.md'))

In [58]:
commit = commits[0]

In [59]:
commit.committer.name

'GitHub Web Flow'

In [60]:
list(commit.get_statuses())

[]

In [61]:
commit.last_modified

'Fri, 04 Mar 2022 23:39:36 GMT'

In [62]:
def commit_date(commit):
    return datetime.datetime.strptime(commit.last_modified, "%a, %d %b %Y %H:%M:%S %Z")
    

In [63]:
commit_date(commit).strftime('%Y-%b-%d')

'2022-Mar-04'

In [64]:
author_and_timestamp = []
prs_for_this_file = set()

for commit in commits:
    author_and_timestamp.append((commit.committer.name, commit_date(commit)))
    prs_for_this_file.update(commit.get_pulls())
    
prs_for_this_file

{PullRequest(title="Allow for PR only edits", number=18),
 PullRequest(title="Release sop drc, remove rdm templates for now", number=14)}

In [ ]:
# Doesn't have the full history of the file.
# Potentially need to use the GraphQL API?

In [65]:
author_and_timestamp = sorted(author_and_timestamp, key=lambda x: x[1])
author_and_timestamp

[('GitHub Web Flow', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('GitHub Web Flow', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('GitHub Web Flow', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36)),
 ('Simon Biggs', datetime.datetime(2022, 3, 4, 23, 39, 36))]

In [66]:
file_creator = author_and_timestamp[0][0]
file_creator

'GitHub Web Flow'

In [67]:
prs_for_this_file = list(commit.get_pulls())
prs_for_this_file

[PullRequest(title="Release sop drc, remove rdm templates for now", number=14)]